In [17]:
from django.db import connection
from django.contrib.auth.models import User
from myboard import models

In [3]:
data = User.objects.all()

In [6]:
# model이 정확히 어떤 클래스 혹은 함수를 가지고 있는지 보고 싶으면 ?? 이용하자
models.Board??

In [8]:
# 실행한 query의 history를 보여주는데 시간순대로 index를 매긴다. / -1은 가장 최근을 의미
# connection.queries[0]
connection.queries[-1]

{'sql': 'SELECT "auth_user"."id", "auth_user"."password", "auth_user"."last_login", "auth_user"."is_superuser", "auth_user"."username", "auth_user"."first_name", "auth_user"."last_name", "auth_user"."email", "auth_user"."is_staff", "auth_user"."is_active", "auth_user"."date_joined" FROM "auth_user" LIMIT 21',
 'time': '0.000'}

In [9]:
# 바람직한 방법은 아님.... use랑 전혀 상관없는 sql문장이 옴... 다음 셀의 방법이 더 정상적임
User.objects.raw('SELECT max(age) FROM myapp_user')

<RawQuerySet: SELECT max(age) FROM myapp_user>

In [10]:
cursor = connection.cursor()
cursor.execute("SELECT max(age) from myapp_user") # sql문장의 결과값을 실행시킴.. but 바로 가져오진 않음.. 그래서 밑의 줄이 더 필요
max_value = cursor.fetchone()[0]  # 데이터 하나만을 가져오는 것이 fetchone

print(max_value)

70


In [26]:
cursor = connection.cursor()
cursor.execute("SELECT * from myboard_board")   # *(all을 의미)는 보안에 취약해서 db의 사용의미를 퇴색시킴....
results = cursor.fetchall()  # 모든 데이터를 다 가져오는 것이 fetchall()  # 그렇기 때문에 데이터가 적을 때 사용해야 한다.

print(type(results[0]))
results[0]

<class 'tuple'>


(1,
 '정남호ㅋㅋ',
 'ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ',
 datetime.datetime(2020, 2, 19, 5, 33, 15, 546472),
 36,
 None,
 'data',
 1)

In [ ]:
# 위의 셀에서 *의 단점 때문에 아래처럼 id, title, cnt만 따로 써서 보여주자
cursor = connection.cursor()
cursor.execute("SELECT id, title, cnt from myboard_board")
results = cursor.fetchall()  # 모든 데이터를 다 가져오는 것이 fetchall()  # 그렇기 때문에 데이터가 적을 때 사용해야 한다.

print(type(results[0]))
results[0]

### 1. DB Browser for SQLite3를 열어보자
이후 `데이터 베이스 열기` 탭을 누르고 Web>mysite>db.sqlite3를 열어서 실행시키고 `데이터보기`탭에서 **myboard_board**을 보자

---

## username이 'home'인 사람이 쓴 모든 글을 보는 방법

2. 방법1) `SQL 실행`탭으로 가서 다음 셀과 같이 입력해보고 결과를 출력해보자. 

In [ ]:
SELECT id, title, cnt
from myboard_board
where author_id = (
SELECT id from auth_user where username ='home')

3. 방법2) **Join**을 통해서 붙여넣는 방법
    -  `SQL 실행`탭으로 가서 다음 셀과 같이 입력해보고 결과를 출력해보자.
    - 이 방법은 Username을 직접 알려주기 때문에 더 <font color=red>**좋은**</font> 방법이다.

In [ ]:
SELECT b.id, title, cnt, username
from myboard_board b, auth_user u
where b.author_id = u.id and username='home'

------

In [135]:
sql = """
SELECT b.id, title, cnt, username
from myboard_board b, auth_user u
where b.author_id = u.id and username='home'
"""

cursor = connection.cursor()
cursor.execute(sql)
r = cursor.fetchone()[0]

In [136]:
print(r)

1


In [137]:
r = cursor.fetchone()
print(r)

(2, 'Benope', 6, 'home')


In [138]:
# 커서는 마지막에 가져올 것이 없다!
r = cursor.fetchall()
print(r)

[(3, '까만색칠판', 2, 'home'), (5, '카페테리아', 2, 'home'), (12, 'asfasfasdfasfdas', 0, 'home'), (13, 'ㄱㄱㄱㄱㄱㄱㄱ', 0, 'home'), (14, 'postit', 0, 'home'), (15, 'adfasfasfasfasdfasfd', 0, 'home'), (16, '1254456956945735', 0, 'home')]


#### 전부다 보고 싶다면 아래 셀처럼 해야함

In [139]:
cursor.execute(sql) # connection.cursor는 이미 위에서 한 번 선언해서 안해도 됌
result = cursor.fetchall() # 이렇게 미리 result를 지정하면 커서의 위치가 처음으로 못 박기 때문에 아래처럼 모든 record들을 언제나 불러올 수 있다.
for r in result :
    print(r)
print('\n')
for r in result :
    print(r[1])

(1, '정남호ㅋㅋ', 36, 'home')
(2, 'Benope', 6, 'home')
(3, '까만색칠판', 2, 'home')
(5, '카페테리아', 2, 'home')
(12, 'asfasfasdfasfdas', 0, 'home')
(13, 'ㄱㄱㄱㄱㄱㄱㄱ', 0, 'home')
(14, 'postit', 0, 'home')
(15, 'adfasfasfasfasdfasfd', 0, 'home')
(16, '1254456956945735', 0, 'home')


정남호ㅋㅋ
Benope
까만색칠판
카페테리아
asfasfasdfasfdas
ㄱㄱㄱㄱㄱㄱㄱ
postit
adfasfasfasfasdfasfd
1254456956945735


---

In [140]:
cursor.description

(('id', None, None, None, None, None, None),
 ('title', None, None, None, None, None, None),
 ('cnt', None, None, None, None, None, None),
 ('username', None, None, None, None, None, None))

In [141]:
#  연습!!
print(zip(['a','b'], (1,2))) # 이건 zip object로만 나옴.... 결과물을 출력하려면 list, dict, set와 같은 것으로 묶어야 함
d = dict( zip(['a','b'], (1,2) ) )
print(d)

{'a': 1, 'b': 2}


In [142]:
d = dict(zip(['id', 'title', 'cnt', 'username'], result[0]))
print(d)

{'id': 1, 'title': '정남호ㅋㅋ', 'cnt': 36, 'username': 'home'}


In [145]:
def dictfetchall(cursor):
    desc = cursor.description
    return[
        dict(zip([col[0] for col in desc], row )) 
        for row in cursor.fetchall()
    ]

In [146]:
cursor = connection.cursor()
cursor.execute('SELECT * from myboard_board')
d = dictfetchall(cursor)
print(d[0]["title"])

정남호ㅋㅋ


---

# Exercise1
1. 게시판 출력(category, username, page) --> 리스트만 출력하면 됌
2. page navigator 안써도 됌
3. models.py에 있는 클래스 쓰지 말야야 하고, views.py에 있는 함수는 이용해도 파일 전체를 이용하면 안됌

In [117]:
from django.db import connection
from django.test import RequestFactory
from django.db import models

rf = RequestFactory()



In [120]:
# 가짜 URL
url = '/myboard/ajaxget?page=1&category="common"&username="home"'
request = rf.get(url)

In [121]:
page = request.GET.get("page", 1)
print(page)
category = request.GET.get("category", "etc")
print(category)
username = request.GET.get("username",'lee')
print(username)

1
"common"
"home"


In [ ]:
# datas = models.Board.objects.all().filter(category='common', page=2, username = 'home')
# page = int(page)
# print(datas)

In [133]:
url = '/myboard/ajaxget?page=1&category="common"&username="home"'
request = rf.get(url)

page = request.GET.get("page", 1)
print(page)
print('\n')

category = request.GET.get("category", "etc")
print(category)
print('\n')

username = request.GET.get("username",'lee')
print(username)

1


"common"


"home"


In [149]:
category = 'common'
page = 2
username = 'home'
sql = f"""
SELECT b.id, title, cnt, username
from myboard_board b, auth_user u
where b.author_id = u.id and username='{username}' and category='{category}'
"""

# print(sql)

cursor = connection.cursor()
cursor.execute(sql)

data = dictfetchall(cursor)
data = data[(page-1)*3: (page*3)]

context = {"data":data}

return render(request, 'myboard/list.html', context)

In [152]:
context = {"data":data}

In [153]:
from myboard import views

response = views.ajaxget(request)
print(response)
print(response.content.decode())

<JsonResponse status_code=200, "application/json">
{"datas": [{"pk": 3, "title": "까만색칠판", "cnt": 2}, {"pk": 12, "title": "asfasfasdfasfdas", "cnt": 0}, {"pk": 13, "title": "ㄱㄱㄱㄱㄱㄱㄱ", "cnt": 0}]}
